# data_prep.py functions

These functions will allow the user to key in their selected cryptocurrencies in their portfolio, and retrieve historical prices from yfinance, a Yahoo! Finance API.

To use these functions in the main script, `import data_prep.py as dp`.

Functions: `dp.input_crypto()`, `dp.get_ticker_data(ticker_list)`

---


## Import libraries and dependencies

In [1]:
import pandas as pd

import datetime

# Import quantstats
import quantstats as qs
qs.extend_pandas()
# Import yfinance
import yfinance as yf


## Function to request user input for the cryptocurrencies they have, and the number of years back for which data is to be fetched.

In [14]:
def get_hypothetical_portfolio():
    '''
    Requests user to key in crypto tickers, and appends to a list.
    '''
    ticker_list = []
    ticker = ""
    print("Please enter the tickers of your cryptocurrencies one by one")
    print("Type 'done' when finished.")
    
    # Keep requesting input from user if user has not typed 'done'
    while ticker.casefold() != "done":
        ticker = str.upper(input("Ticker: "))                           
        if ticker.casefold() != "done":
            ticker_list.append(ticker)
    
    # Adds "-USD" suffix to each crypto ticker for compatibility with yfinance
    ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    return ticker_list

def get_existing_portfolio():
    '''
    Requests user to key in crypto tickers in an existing portfolio, and units of crypto held.
    Records data as a dict
    
    '''
    ticker_dict = {}
    ticker = ""
    print("Please enter the tickers of each cryptocurrency, followed by the number of units you hold:")
    print("Type 'done' when finished.")
    
    # Keep requesting input from user if user has not typed 'done'
    while ticker.casefold() != "done":
        try:
            ticker = str.upper(input("Ticker: "))
            if ticker.casefold()!= "done":
                amount = float(input("No. of units: "))
                ticker_dict[ticker] = amount
        except Exception:
            print("Invalid input. Please ensure you are keying in a valid ticker and a valid number of units.")         
    return ticker_dict

    
def get_investment_amt():
    '''
    Requests investment amount from user. 
    '''
    investment_amount = ""
    while type(investment_amount) is not float:
        try:
            investment_amount = float(input("How much do you wish to invest in total?\n"
                                            "(Please input amount without currency symbol)\n"))
        except Exception:
            print('Error: Please input a numerical value\n'
                 'Remember not to include the currency symbol\n')    
    return investment_amount

        
ticker_list = get_existing_portfolio() 
investment_amount = get_investment_amt()

print(f"--------------------------")                      
print(f"Cryptocurrencies selected:")
print(*ticker_list, sep=', ')                    
print(f"\nInvestment amount:")
print(f"${investment_amount:.2f}")
print(f"--------------------------")  
print(ticker_list)
                        

Please enter the tickers of each cryptocurrency, followed by the number of units you hold:
Type 'done' when finished.


Ticker:  btc
No. of units:  asd


Invalid input. Please ensure you are keying in a valid ticker and a valid number of units.


Ticker:  btc
No. of units:  0.123
Ticker:  eth
No. of units:  2
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 0002


--------------------------
Cryptocurrencies selected:
BTC, ETH

Investment amount:
$2.00
--------------------------
{'BTC': 0.123, 'ETH': 2.0}


In [26]:
# Function to get ticker data. years_back set to 3 by default, given most altcoins don't have a price history dating back more than that. 
def get_ticker_data(ticker_list, years_back = 3):
    ''''
    Iterates through each ticker in user portfolio and fetches OHLCV data from Yahoo Finance API into a pandas dataframe. 
    Also records daily returns for each cryptocurrency as separate columns within the same dataframe.

    Parameters:
    ticker_list (list): A list of cryptocurrency tickers for which data is to be fetched
    years_back (int): Number of years back from the current date, for which data is to be fetched. Set to 3 by default
    '''
    # Initialise dict to record OHLCV data
    d ={}
    # Define start and end dates
    end = datetime.datetime.now()
    start = datetime.datetime.now() - datetime.timedelta(days= 365 * years_back)
    # create empty dataframe
    cryptos_final = pd.DataFrame()
    
    # Get timeframe 
    #interval_list = ['1d', '1w', '1m','1y']
    #interval = str(input(
    #    f"\nPlease select the interval: \n"
    #    f"(Valid intervals: 1d, 1w, 1m, 1y)\n"
    #)
    #              )
                  
    #while interval not in interval_list:
    #    print("\nError: You have selected an invalid interval")
    #    interval = str(input(
    #        f"Please select the interval: \n"
    #        f"(Valid intervals: 1d, 1w, 1m, 1y)\n"
    #    )
    #                  )
                              
    for ticker in ticker_list:
        try:
            # download the crypto price 
            crypto = yf.Ticker(ticker)
            crypto_df = crypto.history(
                start = start, 
                end = end, 
                interval = '1d'
            )
            
            # append the individual crpyto prices 
            if len(crypto_df) == 0:
                None
            else:
                d[(ticker, "open")] = crypto_df['Open']
                d[(ticker, "high")] = crypto_df['High']
                d[(ticker, "low")] = crypto_df["Low"]
                d[(ticker, "close")] = crypto_df["Close"]
                d[(ticker, "volume")] = crypto_df["Volume"]
                d[(ticker, "daily_return")] = crypto_df['Close'].pct_change()
        except Exception:
            None
    
    d = pd.DataFrame(d)
    d = d.dropna()
    return d

# Record portfolio OHLCV as a dataframe.
portfolio_df = get_ticker_data(ticker_list)
